In [2]:
import pandas as pd

df = pd.read_excel('./input/template/Morningstar Performance Template.xls', sheet_name='NAVs', skiprows=7, header=0)

print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Unique Identifier                     0 non-null      object
 1   Fund Class Name                       0 non-null      object
 2   SEDOL                                 0 non-null      object
 3   CUSIP                                 0 non-null      object
 4   MEX ID                                0 non-null      object
 5   NAV/Daily dividend Date               0 non-null      object
 6   NAV/Daily dividend  Currency          0 non-null      object
 7   NAV                                   0 non-null      object
 8   Bid                                   0 non-null      object
 9   Offer                                 0 non-null      object
 10  Market Price                          0 non-null      object
 11  DailyDividend                         0 non-

In [3]:
nav_etpcap2 = pd.read_csv('./input/CAS_Flexfunds_NAV_12202024 ETPCAP2.csv').dropna(how='all', axis=0).dropna(how='all', axis=1)
nav_hfmx = pd.read_csv('./input/CAS_Flexfunds_NAV_12202024 HFMX.csv').dropna(how='all', axis=0).dropna(how='all', axis=1)
nav_iacap = pd.read_csv('./input/CAS_Flexfunds_NAV_12202024 IACAP.csv').dropna(how='all', axis=0).dropna(how='all', axis=1)

nav_df = pd.concat([nav_etpcap2, nav_hfmx, nav_iacap], ignore_index=True)

nav_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   EntityName                 63 non-null     object 
 1   PortfolioName              63 non-null     object 
 2   ISIN                       63 non-null     object 
 3   Account Base Currency      63 non-null     object 
 4   Frequency                  63 non-null     object 
 5   Valuation Period-End Date  63 non-null     object 
 6   Shareholder Equity         63 non-null     object 
 7   NAV                        63 non-null     float64
 8   Outstanding Shares Par     63 non-null     object 
 9   ATWeb Assets               63 non-null     object 
 10  ATWeb Liabilities          63 non-null     object 
 11  Inventory                  63 non-null     object 
dtypes: float64(1), object(11)
memory usage: 6.0+ KB


In [6]:
mapping = {
    'Unique Identifier': 'ISIN',
    'NAV/Daily dividend Date': 'Valuation Period-End Date',
    'NAV': 'NAV'
}

for xls_col, csv_col in mapping.items():
    df[xls_col] = nav_df[csv_col]


import openpyxl
# df.to_excel('./Morningstar Performance Template.xlsx', sheet_name='NAVs', index=False)
wb = openpyxl.load_workbook('./input/template/Morningstar Performance Template.xls')
ws = wb['NAVs']

for i, row in enumerate(df.values):
    for j, value in enumerate(row):
        # Add 8 to account for the header rows
        ws.cell(row=i+8, column=j+1, value=value)

# Save the workbook with preserved formatting
wb.save('./Morningstar Performance Template_updated.xlsx')

InvalidFileException: openpyxl does not support the old .xls file format, please use xlrd to read this file, or convert it to the more recent .xlsx file format.

In [7]:
import pandas as pd
import xlrd
import xlutils.copy

# Read the template to get structure and preserve formatting
template_path = './input/template/Morningstar Performance Template.xls'
wb = xlrd.open_workbook(template_path, formatting_info=True)
template_sheet = wb.sheet_by_name('NAVs')

# Create a copy of the workbook to modify
wb_output = xlutils.copy.copy(wb)
sheet_output = wb_output.get_sheet('NAVs')

# Read NAV data
nav_etpcap2 = pd.read_csv('./input/CAS_Flexfunds_NAV_12202024 ETPCAP2.csv').dropna(how='all', axis=0).dropna(how='all', axis=1)
nav_hfmx = pd.read_csv('./input/CAS_Flexfunds_NAV_12202024 HFMX.csv').dropna(how='all', axis=0).dropna(how='all', axis=1)
nav_iacap = pd.read_csv('./input/CAS_Flexfunds_NAV_12202024 IACAP.csv').dropna(how='all', axis=0).dropna(how='all', axis=1)

nav_df = pd.concat([nav_etpcap2, nav_hfmx, nav_iacap], ignore_index=True)

# Get column indices from template (row 7 contains headers)
header_row = 7
col_indices = {}
for col_idx in range(template_sheet.ncols):
    header_value = template_sheet.cell_value(header_row, col_idx)
    col_indices[header_value] = col_idx

# Mapping of columns
mapping = {
    'Unique Identifier': 'ISIN',
    'NAV/Daily dividend Date': 'Valuation Period-End Date',
    'NAV': 'NAV'
}

# Update the data starting from row 8
for i, row in nav_df.iterrows():
    row_idx = i + 8  # Start from row 8
    for template_col, nav_col in mapping.items():
        col_idx = col_indices[template_col]
        value = row[nav_col]
        sheet_output.write(row_idx, col_idx, value)

# Save the updated workbook
wb_output.save('Morningstar Performance Template_updated.xls')

ModuleNotFoundError: No module named 'xlutils'